In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.discrete.count_model as smdc
import patsy

In [2]:
fake_cres=pd.read_csv("fake_cres.csv").drop("Unnamed: 0",axis=1)

In [3]:
fake_cres


,CRE,Cell_type,replicate_ID,umi_count
0,nobody,brain,1,0
1,nobody,brain,1,0
2,nobody,brain,1,0
3,nobody,brain,1,0
4,nobody,brain,1,0
...,...,...,...,...
14307,neurogene,blood,3,7
14308,neurogene,blood,3,26
14309,neurogene,blood,3,7
14310,neurogene,blood,3,15


In [4]:
fake_cres_munged=fake_cres
fake_cres_munged["replicate_ID"]=fake_cres_munged["replicate_ID"].map({1:"rep1",2:"rep2",3:"rep3"})

Some of these are strings or ints when they should be categorical...

We can either cast to categorical in the dataframe, or specify that the values should be categorical in the formula (for items in the formula) or one-hot with `pd.dummies` for the statsmodels interface.

A couple different ways of stipulating the model. 
- String formula can't use the pipe operator to pass a second formula 

In [5]:
fake_cres_munged

,CRE,Cell_type,replicate_ID,umi_count
0,nobody,brain,rep1,0
1,nobody,brain,rep1,0
2,nobody,brain,rep1,0
3,nobody,brain,rep1,0
4,nobody,brain,rep1,0
...,...,...,...,...
14307,neurogene,blood,rep3,7
14308,neurogene,blood,rep3,26
14309,neurogene,blood,rep3,7
14310,neurogene,blood,rep3,15


In [6]:

y, X = patsy.dmatrices("umi_count ~ C(CRE)*C(Cell_type)-1",
                        fake_cres_munged, return_type='dataframe')
Z = patsy.dmatrix("C(replicate_ID)", fake_cres_munged, return_type='dataframe')

zinb_model = smdc.ZeroInflatedNegativeBinomialP(y, X, exog_infl=Z)

n_count_params = zinb_model.exog.shape[1]      # Count model parameters
n_infl_params = zinb_model.exog_infl.shape[1]    # Inflation model parameters
n_total = n_count_params + n_infl_params + 1 # adding 1 for alpha
start_params = np.full(n_total, 0.1)

zinb_result = zinb_model.fit(start_params=start_params,maxiter=1000)

Optimization terminated successfully.
         Current function value: 1.483284
         Iterations: 84
         Function evaluations: 85
         Gradient evaluations: 85


https://stats.stackexchange.com/questions/284911/type-i-and-type-ii-negative-binomial-distribution-in-zero-inflated-negative-bino

https://www.statsmodels.org/stable/generated/statsmodels.discrete.count_model.ZeroInflatedNegativeBinomialP.html

https://www.statsmodels.org/dev/generated/statsmodels.discrete.count_model.ZeroInflatedNegativeBinomialP.from_formula.html#statsmodels.discrete.count_model.ZeroInflatedNegativeBinomialP.from_formula

In [8]:
print(type(zinb_result))
zinb_result.summary()

<class 'statsmodels.discrete.count_model.ZeroInflatedNegativeBinomialResultsWrapper'>


<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                         umi_count   No. Observations:                14312
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    14302
Method:                                      MLE   Df Model:                            9
Date:                           Mon, 24 Feb 2025   Pseudo R-squ.:                  0.1598
Time:                                   14:17:24   Log-Likelihood:                -21229.
converged:                                  True   LL-Null:                       -25265.
Covariance Type:                       nonrobust   LLR p-value:                     0.000
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
inflate_Intercept                             1.3554      0.037     36.249      0.000       1.282       1.429
inflate_C(replicate_ID)[T.rep2]              -1.3273      0.048    -27.603      0.000      -1.422      -1.233
inflate_C(replicate_ID)[T.rep3]               0.8887      0.063     14.102      0.000       0.765       1.012
C(CRE)[everybody]                             4.6398      0.028    167.429      0.000       4.586       4.694
C(CRE)[neurogene]                             2.7408      0.030     90.617      0.000       2.681       2.800
C(CRE)[nobody]                                0.6659      0.052     12.869      0.000       0.565       0.767
C(CRE)[redgene]                               4.6594      0.027    171.854      0.000       4.606       4.713
C(CRE)[somebody]                              2.5254      0.030     82.913      0.000       2.466       2.585
C(Cell_type)[T.brain]                         0.0399      0.040      0.999      0.318      -0.038       0.118
C(CRE)[T.neurogene]:C(Cell_type)[T.brain]     1.7868      0.057     31.104      0.000       1.674       1.899
C(CRE)[T.nobody]:C(Cell_type)[T.brain]       -0.6792      0.094     -7.252      0.000      -0.863      -0.496
C(CRE)[T.redgene]:C(Cell_type)[T.brain]      -1.2739      0.056    -22.549      0.000      -1.385      -1.163
C(CRE)[T.somebody]:C(Cell_type)[T.brain]     -0.2638      0.061     -4.352      0.000      -0.383      -0.145
alpha                                         0.2868      0.008     35.107      0.000       0.271       0.303
=============================================================================================================
"""

# Recapitulation of $\theta$ (constant in mean-variance quadratic)

First, let's examine theta. That is, the theta in $\sigma^2=\mu+\mu^2/\theta$. 

From our generation step, we know the true $\theta$ value to be 0.3.

Statsmodels seems to call it alpha, but regardless, it's quite close to the real value.

In [8]:
zinb_result.params["alpha"]

np.float64(0.2867779982959691)

# Recapitulating $\mu$ values

In [9]:
#we begin by getting all combinations of the predictors (which are of course all categorical) present in the data. 
minimal_nb_design = X.drop_duplicates()
minimal_nb_design

,C(CRE)[everybody],C(CRE)[neurogene],C(CRE)[nobody],C(CRE)[redgene],C(CRE)[somebody],C(Cell_type)[T.brain],C(CRE)[T.neurogene]:C(Cell_type)[T.brain],C(CRE)[T.nobody]:C(Cell_type)[T.brain],C(CRE)[T.redgene]:C(Cell_type)[T.brain],C(CRE)[T.somebody]:C(Cell_type)[T.brain]
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
440,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
904,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1355,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1798,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2263,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2762,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3266,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3767,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4262,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#let's look at our paramaters (fitted betas)
zinb_result.params

inflate_Intercept                            1.355423
inflate_C(replicate_ID)[T.rep2]             -1.327281
inflate_C(replicate_ID)[T.rep3]              0.888705
C(CRE)[everybody]                            4.639833
C(CRE)[neurogene]                            2.740756
C(CRE)[nobody]                               0.665935
C(CRE)[redgene]                              4.659436
C(CRE)[somebody]                             2.525434
C(Cell_type)[T.brain]                        0.039876
C(CRE)[T.neurogene]:C(Cell_type)[T.brain]    1.786778
C(CRE)[T.nobody]:C(Cell_type)[T.brain]      -0.679159
C(CRE)[T.redgene]:C(Cell_type)[T.brain]     -1.273865
C(CRE)[T.somebody]:C(Cell_type)[T.brain]    -0.263831
alpha                                        0.286778
dtype: float64

In [11]:
#extract the inflation parameters into one variable (for later use)
#and the negative-binomial parameters into a different variable (for immediate use)
inflation_betas=zinb_result.params[0:n_infl_params]
nb_betas=zinb_result.params[n_infl_params:n_count_params + n_infl_params]
#possibly a more robust way of doing this would be on the basis of names
#So long as none of the nb predictors have "inflate" in their names...

In [12]:
#make sure they are in the same order...
all(nb_betas.keys()==minimal_nb_design.columns)

True

In [13]:
#get the linear version of the predictors with a dot-product
linear_predictors=minimal_nb_design.dot(nb_betas)
#undo the log-link
expected_values=np.exp(linear_predictors)

In [14]:
#make and display a little dataframe.
#hard-coding the names for this demo, we could easially un-one-hot 
#or make the minimal_nb_design mat from scratch for more complicated cases
recapitulated_nb_rate=pd.DataFrame({"cre":["nobody","somebody","everybody","redgene","neurogene","nobody","somebody","everybody","redgene","neurogene"],
    "cell_type":["brain"]*5+["blood"]*5,
    "expected_rate":expected_values})
recapitulated_nb_rate

,cre,cell_type,expected_rate
0,nobody,brain,1.027010
440,somebody,brain,9.988947
904,everybody,brain,107.738697
1355,redgene,brain,30.736365
1798,neurogene,brain,96.294471
2263,nobody,blood,1.946309
2762,somebody,blood,12.496317
3266,everybody,blood,103.527025
3767,redgene,blood,105.576498
4262,neurogene,blood,15.498705


Comparing to real data...

In [15]:
REAL_data = {
    "CRE": ["nobody", "somebody", "everybody", "redgene", "neurogene", "nobody", "somebody", "everybody", "redgene", "neurogene"],
    "Cell-type": ["brain", "brain", "brain", "brain", "brain", "blood", "blood", "blood", "blood", "blood"],
    "mean": [1, 10, 114, 30, 99, 2, 12, 109, 112, 16]
}

# Creating the dataframe
pd.DataFrame(REAL_data)

,CRE,Cell-type,mean
0,nobody,brain,1
1,somebody,brain,10
2,everybody,brain,114
3,redgene,brain,30
4,neurogene,brain,99
5,nobody,blood,2
6,somebody,blood,12
7,everybody,blood,109
8,redgene,blood,112
9,neurogene,blood,16


Works very well.

# Recapitulation of zero-inflation parameter

In [35]:
minimal_dropout_design=Z.drop_duplicates()
minimal_dropout_design

,Intercept,C(replicate_ID)[T.rep2],C(replicate_ID)[T.rep3]
0,1.0,0.0,0.0
4761,1.0,1.0,0.0
9536,1.0,0.0,1.0


In [38]:
linear_dropout_pred=minimal_dropout_design.to_numpy() @ inflation_betas
linear_dropout_pred

array([1.35542266, 0.02814137, 2.24412758])

In [42]:
recovered_dropout=1/(1+np.exp(-linear_dropout_pred))
#top to bottom in the matrix are : intercept only (rep1 by process of elim), rep2, rep3
pd.DataFrame({'replicate':["rep1","rep2","rep3"],'dropout':recovered_dropout})

,replicate,dropout
0,rep1,0.795015
1,rep2,0.507035
2,rep3,0.904143


Compared to real values (.8, .5, .9), that's pretty good!

# Unsort

In [60]:
class ZINBFixedAlpha(smdc.ZeroInflatedNegativeBinomialP):
    def __init__(self, endog, exog, exog_infl, fixed_alpha, **kwargs):
        self.fixed_alpha = fixed_alpha  # Save the fixed value of alpha
        super(ZINBFixedAlpha, self).__init__(endog, exog, exog_infl=exog_infl, **kwargs)

    def loglike(self, params):
        """
        Override the log-likelihood so that the overdispersion parameter (alpha)
        is fixed at self.fixed_alpha.

        In the original model, the parameter vector is assumed to be:
            [beta (count params), gamma (inflation params), alpha]

        Here we rebuild the full parameter vector by appending fixed_alpha.
        """
        # Determine the number of parameters in each part:
        k_count = self.exog.shape[1]
        k_infl = self.exog_infl.shape[1] if self.exog_infl is not None else 0

        # Ensure that the incoming params vector has the correct length:
        # It should have length = k_count + k_infl.
        if len(params) != k_count + k_infl:
            raise ValueError("Length of params should be equal to k_count + k_infl.")

        # Build the full parameter vector by appending fixed_alpha:
        full_params = np.r_[params, self.fixed_alpha]

        # Now call the original loglike with the full parameter vector.
        return super(ZINBFixedAlpha, self).loglike(full_params)

    def loglikeobs(self, params):
        """
        Override loglikeobs similarly if needed. This ensures that any calls to the
        per-observation likelihood also use the fixed alpha.
        """
        k_count = self.exog.shape[1]
        k_infl = self.exog_infl.shape[1] if self.exog_infl is not None else 0
        if len(params) != k_count + k_infl+1:
            raise ValueError("Length of params should be equal to k_count + k_infl+1.")
        full_params = np.r_[params, self.fixed_alpha]
        return super(ZINBFixedAlpha, self).loglikeobs(full_params)

In [61]:
zinb_constrained = ZINBFixedAlpha(y, X, exog_infl=Z,fixed_alpha=0.3)
#zinb_constrained.fix_params()
start_params_reduced= np.full(X.shape[1]+Z.shape[1], 0.1)
zinb_constrained.fit(start_params=start_params_reduced,maxiter=1000)

ValueError: shapes (14312,10) and (9,) not aligned: 10 (dim 1) != 9 (dim 0)